In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.activations import *
from tensorflow.keras.regularizers import *
from tensorflow.keras import Input

from typing import Union , Tuple , Dict , List

In [ ]:
def conv2d_bn_activation(input_tensor: tf.float32,
                         filters: int,
                         kernel_size: int,
                         strides: int = 2,
                         padding: str = 'same') -> tf.float32:
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=(strides, strides),
               padding=padding)(input_tensor)
    x = BatchNormalization()(x)
    return LeakyReLU(alpha=0.1)(x)



def residual_block(input_tensor: tf.float32,
                   filters: int) -> tf.float32 :
    x = conv2d_bn_activation(input_tensor=input_tensor,
                             filters=filters,
                             kernel_size=1,
                             strides=1)
    x = conv2d_bn_activation(input_tensor=x,
                             filters=2 * filters,
                             kernel_size=3,
                             strides=1)
    return Add()([input_tensor, x])


def conv_residual(num_repeat: int,
                  input_tensor: tf.float32,
                  filters: int) -> tf.float32:
    x = conv2d_bn_activation(input_tensor=input_tensor,
                             filters=2*filters,
                             kernel_size=1,
                             strides=1)
    for _ in range(num_repeat):
        x = residual_block(input_tensor=x,
                           filters=filters)
    return x

 

def DarkNet_53(height: int,
               width: int,
               color_channels: int,
               num_classes: int) -> tf.float32:
    input_shape = (height, width, color_channels)
    inputs = Input(shape=input_shape)

    x = conv2d_bn_activation(input_tensor=inputs,
                             filters=32,
                             kernel_size=3,
                             strides=1)  # feature_maps = 32
    x = conv2d_bn_activation(input_tensor=x,
                             filters=64,
                             kernel_size=3)  # feature_maps = 64
    x = conv_residual(num_repeat=1,
                      input_tensor=x,
                      filters=32)  # feature_maps = 64
    x = conv_residual(num_repeat=2,
                      input_tensor=x,
                      filters=128)  # feature_maps = 256
    x = conv_residual(num_repeat=8,
                      input_tensor=x,
                      filters=128)  # feature_maps = 512
    x = conv_residual(num_repeat=8,
                      input_tensor=x,
                      filters=256)  # feature_maps = 1024
    x = conv_residual(num_repeat=4,
                      input_tensor=x,
                      filters=512)  # feature_maps = 1024
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax', name='output')(x)
    return Model(inputs=inputs, outputs=outputs, name='Darknet53')